# PyShEx Shape inconsistency detection
If `satisfiesShape` is recursively called with the same focus node and shape definition, it:

1) Conditionally asserts that the result is `True`.  If this asserts results in the outermost evaluation to also be `True`, the focus node is reported as conforming to the shape definition.

2) If assertion 1 results in an evaluation of `False`, it then conditionally asserts that the result is `False`.  If this results in the entire evaluation being `False`, the focus node is reported as not conforming to the shape definition.  

3) Otherwise, the focus node is reported as "not conforming" with the reason being that the definition is inconsistent.  (Quotes because it isn't actually possible to determine conformance to an inconsistent shape, but the return structure doesn't support a third alternative)

In [1]:
!pip install PyShEx --upgrade -q

In [2]:
from rdflib import Graph, Namespace
from pyshex import ShExEvaluator


EX = Namespace("http://a.example/")

In [3]:
shex = """<http://a.example/S> {<http://a.example/p> @<http://a.example/S>}"""
g = Graph()
g.add((EX.x, EX.p, EX.x))
e = ShExEvaluator(rdf=g, schema=shex, focus=EX.x, start=EX.S, debug=False)
print(e.evaluate()[0].result)

True


In [4]:
shex = """<http://a.example/S> {<http://a.example/p> not @<http://a.example/S>}"""
g = Graph()
g.add((EX.x, EX.p, EX.x))
e = ShExEvaluator(rdf=g, schema=shex, focus=EX.x, start=EX.S, debug=False)
rslt = e.evaluate()
print(f"{rslt[0].result}: {rslt[0].reason}")

False:   Testing <http://a.example/x> against shape http://a.example/S
    Testing <http://a.example/x> against shape http://a.example/S
      http://a.example/S: Inconsistent recursive shape reference
